<div class="alert alert-block alert-success">
    <b><center>PostgreSQL</center></b>
    <b><center>Python을 이용한 RDB 프로그래밍 기본</center></b>
</div>

In [232]:
import psycopg2
import pandas as pd
import numpy as np

from psycopg2 import pool
import mybatis_mapper2sql
from multiprocessing import Process

In [224]:
class PGManager(object):
    def __new__(self):
        if not hasattr(self, 'instance'):
            self.instance = super(PGManager, self).__new__(self)
            self.__connection_pool = None
        return self.instance
    
    def createPool(self, user, passwd, db, host, minCon=5, maxCon=20, port=5432):
        self.close()
        self.__connection_pool = pool.SimpleConnectionPool(
            minCon, maxCon, user = user, password=passwd,
            host=host, port=port, database=db)
    
    def close(self):
        if (self.__connection_pool):
            self.__connection_pool.closeall()
    
    def setMapper(self, xmlMapperFile):
        self._mapper, _ = mybatis_mapper2sql.create_mapper(xml=xmlMapperFile)
    
    def query(self, mapId, params=None):
        connection = self.__connection_pool.getconn()
        cursor = connection.cursor()
        statement = mybatis_mapper2sql.get_child_statement(self._mapper, mapId)
        cursor.execute(statement, params)
        records = cursor.fetchall()
        cursor.close()
        self.__connection_pool.putconn(connection)
        
        return records
    
    def execute(self, mapId, params=None):
        connection = self.__connection_pool.getconn()
        cursor = connection.cursor()
        statement = mybatis_mapper2sql.get_child_statement(self._mapper, mapId)
        cursor.execute(statement, params)
        cursor.close()
        connection.commit()
        self.__connection_pool.putconn(connection)
    
    def executeMany(self, mapId, paramsList):
        connection = self.__connection_pool.getconn()
        cursor = connection.cursor()
        statement = mybatis_mapper2sql.get_child_statement(self._mapper, mapId)
        cursor.executemany(statement, paramsList)
        cursor.close()
        connection.commit()
        self.__connection_pool.putconn(connection)
    
    def asyncExecuteMany(self, mapper, mapId, paramsList):
        p = Process(target=self.executeMany, args=(mapId, paramsList))
        p.start()
        p.join()

In [225]:
db = PGManager()

In [226]:
db.createPool("postgres", "postgres", "postgres", "192.168.0.200")
db.setMapper("mapper.xml")

In [227]:
db.execute("initDB")

# xls를 DB에 저장.

In [228]:
df = pd.read_excel("유가증권.xlsx", encoding="utf-8", dtype={"종목코드":str})
df.fillna("", inplace = True) 

In [229]:
df.head()

,회사명,종목코드,업종,주요제품,상장일,결산월,대표자명,홈페이지,지역
0,에스케이바이오팜,326030,자연과학 및 공학 연구개발업,신경질환 및 정신질환을 포함한 중추신경 관련 신약 개발,2020-07-02,12월,조정우,,경기도
1,대덕전자,353200,전자부품 제조업,"PCB(Printed Circuit Board, 인쇄회로기판)의 제조 및 판매",2020-05-21,12월,신영환,http://www.daeduck.com,경기도
2,케이씨씨글라스,344820,유리 및 유리제품 제조업,판유리 등,2020-01-21,12월,김내환,http://www.kccglass.co.kr,서울특별시
3,NH프라임리츠,338100,부동산 임대 및 공급업,부동산 투자,2019-12-05,11월,권오성,http://www.nhreits.com,서울특별시
4,센트랄모텍,308170,자동차 신품 부품 제조업,"이너레이스, 오디 클러치보스, 로터샤프트, 슬리브기어, 컨트롤 암, 엑슬",2019-11-25,12월,"이종철, 강태룡",http://www.centralmotek.co.kr,울산광역시


In [230]:
params = []
for item in df.to_numpy():
    params.append((str(item[1]), item[0], item[2], item[3], str(item[4]), item[5][:-1], item[6], item[7], item[8], False))

In [231]:
db.executeMany("inCorporation", params)

# 각종 쿼리

In [233]:
a = "20180530"
b = "20180601"

In [235]:
d = datetime.datetime.strptime('2018-05-19', '%Y-%m-%d')

False

In [236]:
a = {"1":"a"}

In [237]:
a

{'1': 'a'}

In [238]:
a.clear()

In [239]:
a

{}